<a href="https://colab.research.google.com/github/girijeshcse/car_finder/blob/ambika/notebooks/cars_finder_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

In [ ]:
import random
import os 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
  
from PIL import Image
import numpy as np

import tensorflow as tf
import os
import pandas as pd

random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
ls "/content/drive/MyDrive/Dataset/Car Images/"

In [ ]:
dataset_loc="/content/drive/MyDrive/"
train_annotation = pd.read_csv(dataset_loc+"/Dataset/Annotations/Train Annotations.csv")
train_annotation.info()

In [ ]:
entry = train_annotation.loc[train_annotation['Image Name'] == '02256.jpg'].values[0]
#dim_analysis.loc[dim_analysis['dim3'] !=3 ]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
entry[1]

In [ ]:
train_annotation.rename(columns = {'Bounding Box coordinates':'x0', 'Unnamed: 2':'yo', 'Unnamed: 3':'x1', 'Unnamed: 4':'y1'}, inplace = True)
   

In [ ]:
train_annotation.loc[train_annotation['Image Name'] == '00134.jpg']

In [ ]:
train_annotation

In [ ]:


train_images = pd.DataFrame(columns=['image','image_matrix','filename','car', 'x0', 'y0', 'x1', 'y1', 'label'])
dim_analysis = pd.DataFrame(columns=['filename', 'dim1', 'dim2', 'dim3'])


base=dataset_loc+"/Dataset/Car Images/Train Images"
newsize = (128, 128)
reshape_size = 128*128*3
size_dim=128


index = 0
for dirs in os.listdir(base):
    full_dir=base+"/"+dirs
    
    if os.path.isfile(full_dir):
      continue
    for file in os.listdir(os.path.join(full_dir)):
      
      if os.path.isfile(os.path.join(full_dir, file)):
        filename=full_dir+"/"+file
        im = Image.open(filename, mode='r')
        dim_path=dirs+"/"+filename
        image_width = im.size[1]
        image_height = im.size[0]
        dim_analysis = dim_analysis.append({'filename': filename, 'dim1':image_height, 'dim2':image_width}, ignore_index = True)
  
            #print(im.size[0])
        im = im.resize(newsize)
        pix_val = np.array(list(im.getdata()))
        #print("\t \t", len(pix_val.shape))
        if ((len(pix_val.shape) >= 2) and (pix_val.shape[0]*pix_val.shape[1] == reshape_size)):
          bound_box = train_annotation.loc[train_annotation['Image Name'] == file].values[0]
          x0=bound_box[1]
          x0 = int((x0 / image_width) * size_dim) 

          y0=bound_box[2]
          y0 = int((y0 / image_height ) * size_dim)

          x1=bound_box[3]
          x1 = int((x1 / image_width)  * size_dim)

          y1=bound_box[4]
          y1 = int((y1 / image_height) * size_dim)
          train_images = train_images.append({'image': pix_val.reshape(reshape_size), 'image_matrix': pix_val, 'filename':full_dir+"/"+file, 'car':dirs, 'x0': x0, 'y0':y0, 'x1':x1, 'y1':y1, 'label':bound_box[5]}, ignore_index = True)
          


In [ ]:
train_images

In [ ]:
dim_analysis.loc[0, 'filename']

In [ ]:
print("Various different dimensions of the image: \n ", np.unique(dim_analysis['dim1']))

In [ ]:
import seaborn as sns
print("Dimension Analysis:\n")
val_count = dim_analysis['dim1'].value_counts()
print(val_count)


In [ ]:

sns.histplot(dim_analysis['dim1'])

In [ ]:
val_dict = dict(zip(train_images.car, train_images.label))

In [ ]:
val_dict

In [ ]:

print("car counts:\n")
val_count = train_images['car'].value_counts()
print(val_count)

In [ ]:
sns.histplot(train_images['car'])

In [ ]:
plt.imshow(im)
plt.show()

In [ ]:
# few statistical analysis
train_images.loc[train_images['car'] == 'GMC Savana Van 2012']


In [ ]:
len_train = train_images.shape[0]

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Some random images
for i in range( 2, 1000, 100):
    print("=============\n", train_images.loc[i,'car'])
    plt.imshow(train_images.loc[i, 'image'].reshape(size_dim, size_dim, 3))
    plt.show()

In [ ]:
dim_analysis

In [ ]:
def normalising_box(ind):
  test_img = train_images.loc[ind, 'filename']
  test_dim = dim_analysis.loc[dim_analysis['filename'] == test_img].values[0]
  image_width = test_dim[2]
  image_height = test_dim[1]
 
  IMAGE_SIZE=128
  x0=y_train[ind][0]
  y0=y_train[ind][1]

  x1=y_train[ind][2]
  y1=y_train[ind][3]

  print(x0, y0, x1, y1)
  
  x0 = int((x0 / image_width) * IMAGE_SIZE) # Scale the BBox
  y0 = int((y0 / image_height ) * IMAGE_SIZE)

  x1 = int((x1 / image_width)  * IMAGE_SIZE)
  y1 = int((y1 / image_height) * IMAGE_SIZE)
  print(x0, y0, x1, y1)
  #return x0, y0, x1, y1


In [ ]:
import matplotlib.pyplot as plt
#Extract the (flattened) data and class



from tensorflow.keras.applications.mobilenet import preprocess_input

#x_train = train_images['image_matrix']

x_train = np.zeros((len_train, size_dim, size_dim, 3))
y_train = np.zeros((len_train, 4))
y_train_class = train_images['label']
#y_train = train_images[5:7]
for i in range(len(train_images)) :
  x_train[i] = preprocess_input(np.array(train_images.loc[i,'image_matrix'].reshape(size_dim, size_dim, 3), dtype=np.float32))
  y_train[i] = train_images.loc[i, 'x0':'y1']

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def patch_fn (ind):
  
  
  # Create figure and axes
  fig,ax = plt.subplots(1)
  x0=y_train[ind][0]
  y0=y_train[ind][1]

  x1=y_train[ind][2]
  y1=y_train[ind][3]
  print("Normalised Bounding box: ", x0, y0, x1, y1)
  # Display the image
  ax.imshow(train_images.loc[ind,'image_matrix'].reshape(size_dim,size_dim,3))

  # Create a Rectangle patch
  rect = patches.Rectangle((x0, y0), x1 , y1, linewidth=2, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
  ax.add_patch(rect)

  plt.show()



In [ ]:
#Some random bounding box examples
patch_fn(50)
#normalising_box(50)

In [ ]:
patch_fn(200)

In [ ]:
patch_fn(4000)

In [ ]:
patch_fn(6123)

In [ ]:
x_train.shape, y_train.shape, y_train_class.shape

In [ ]:
x_train[25], y_train[25], y_train_class[25]

In [ ]:
del train_images, dim_analysis, train_annotation
#gc.collect()

Considering VGG16 to do object detection region and if needed SVM on the flatten layer to detect the class(196 class)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
y_train

In [ ]:
dataset_loc="/content/drive/MyDrive/"
test_annotation = pd.read_csv(dataset_loc+"/Dataset/Annotations/Test Annotation.csv")
test_annotation.info()

In [ ]:
test_annotation.rename(columns = {'Bounding Box coordinates':'x0', 'Unnamed: 2':'yo', 'Unnamed: 3':'x1', 'Unnamed: 4':'y1'}, inplace = True)
   

In [ ]:
#TEst images




test_images = pd.DataFrame(columns=['image','image_matrix','filename','car', 'x0', 'y0', 'x1', 'y1', 'label'])
dim_analysis_test = pd.DataFrame(columns=['filename', 'dim1', 'dim2', 'dim3'])


base=dataset_loc+"/Dataset/Car Images/Test Images"
newsize = (128, 128)
reshape_size = 128*128*3
size_dim=128


index = 0
for dirs in os.listdir(base):
    full_dir=base+"/"+dirs
    
    if os.path.isfile(full_dir):
      continue
    for file in os.listdir(os.path.join(full_dir)):
      
      if os.path.isfile(os.path.join(full_dir, file)):
        filename=full_dir+"/"+file
        im = Image.open(filename, mode='r')
        dim_path=dirs+"/"+filename
        image_width = im.size[1]
        image_height = im.size[0]
        dim_analysis_test = dim_analysis_test.append({'filename': filename, 'dim1':image_height, 'dim2':image_width}, ignore_index = True)

        im = im.resize(newsize)
        pix_val = np.array(list(im.getdata()))
        if ((len(pix_val.shape) >= 2) and (pix_val.shape[0]*pix_val.shape[1] == reshape_size)):
          bound_box = test_annotation.loc[test_annotation['Image Name'] == file].values[0]
          x0=bound_box[1]
          x0 = int((x0 / image_width) * size_dim) 

          y0=bound_box[2]
          y0 = int((y0 / image_height ) * size_dim)

          x1=bound_box[3]
          x1 = int((x1 / image_width)  * size_dim)

          y1=bound_box[4]
          y1 = int((y1 / image_height) * size_dim)
          test_images = test_images.append({'image': pix_val.reshape(reshape_size), 'image_matrix': pix_val, 'filename':full_dir+"/"+file, 'car':dirs, 'x0': x0, 'y0':y0, 'x1':x1, 'y1':y1, 'label':bound_box[5]}, ignore_index = True)
          


In [ ]:
test_images

In [ ]:
import seaborn as sns
print("Dimension Analysis:\n")
val_count = dim_analysis_test['dim1'].value_counts()
print(val_count)

sns.histplot(dim_analysis_test['dim1'])

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Some random images
for i in range( 2, 1000, 100):
    print("=============\n", test_images.loc[i,'car'])
    plt.imshow(test_images.loc[i, 'image'].reshape(size_dim, size_dim, 3))
    plt.show()

In [ ]:
len_test = test_images.shape[0]

In [ ]:
import matplotlib.pyplot as plt
#Extract the (flattened) data and class

from tensorflow.keras.applications.mobilenet import preprocess_input


x_test = np.zeros((len_test, size_dim, size_dim, 3))
y_test = np.zeros((len_test, 4))
y_test_class = test_images['label']
#y_train = train_images[5:7]
for i in range(len(test_images)) :
  x_test[i] = preprocess_input(np.array(test_images.loc[i,'image_matrix'].reshape(size_dim, size_dim, 3), dtype=np.float32))
  y_test[i] = test_images.loc[i, 'x0':'y1']
  

In [ ]:
y_test_class

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

In [ ]:
del test_images, dim_analysis_test, test_annotation

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import argparse
import cv2



from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape
IMAGE_SIZE=128


In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape

ALPHA = 1.0 # Width hyper parameter for MobileNet (0.25, 0.5, 0.75, 1.0). Higher width means more accurate but slower


model = MobileNet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA) # Load pre-trained mobilenet
for layer in model.layers:
    layer.trainable = True

   

In [ ]:
# Add new top layer which is a conv layer of the same size as the previous layer so that only 4 coords of BBox can be output
x0 = model.layers[-1].output
x1 = Conv2D(4, kernel_size=4, name="coords")(x0)
x2 = Reshape((4,))(x1)
model_region = Model(inputs=model.input, outputs=x2)
model_region.summary()



In [ ]:
model_region.summary()

In [ ]:
def IOU(y_true, y_pred):
    intersections = 0
    unions = 0
    # set the types so we are sure what type we are using

    gt = y_true
    pred = y_pred
    # Compute interection of predicted (pred) and ground truth (gt) bounding boxes
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height

    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection

    # Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
      if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
        intersections += intersection[j]
        unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + tensorflow.keras.backend.epsilon()), 4)
    # This must match the type used in py_func
    iou = iou.astype(np.float32)
    return iou

def IoU(y_true, y_pred):
    iou = tensorflow.py_function(IOU, [y_true, y_pred], Tout=tensorflow.float32)
    return iou


In [ ]:
model_region.compile(loss="mean_squared_error", optimizer="adam", metrics=[IoU]) # Regression loss is MSE



In [ ]:
callback = tensorflow.keras.callbacks.EarlyStopping(monitor='IoU', patience=10, min_delta=0.001)

# Fit the model
model_region.fit(x_train, y_train,validation_data =(x_test, y_test), epochs=200, batch_size=32, callbacks=[callback])
#, callbacks=[callback])
#, validation_data=(X_val, y_val)

In [ ]:
model_region.evaluate(x_train, y_train)

In [ ]:
model_region.evaluate(x_test, y_test)

In [ ]:
x_train[0].shape

In [ ]:
model_region.predict(x_train[0].reshape(1, 128, 128, 3))

In [ ]:
y_train[0]

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_class=to_categorical(y_train_class)

y_test_class=to_categorical(y_test_class)

In [ ]:
y_test_class[250]

In [ ]:
y_train_class.shape, y_test_class.shape

In [ ]:
base_model_resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(128,128,3),classes=y_test_class.shape[1])
base_model_vgg16 = VGG16(include_top=False,weights='imagenet',input_shape=(128,128,3),classes=y_test_class.shape[1])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout, BatchNormalization, Dense
for layer in base_model_vgg16.layers:
  layer.trainable = False

#Defining and Adding layers
model_class=Sequential()
#Add the Dense layers along with activation and batch normalization
model_class.add(base_model_vgg16)
model_class.add(Flatten())

model_class.summary()



In [ ]:

#Add the Dense layers along with activation and batch normalization
model_class.add(Dense(2048,activation=('relu'),input_dim=8192))
model_class.add(Dense(1024,activation=('relu'))) 
model_class.add(Dropout(.4))
model_class.add(Dense(512,activation=('relu'))) 
model_class.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_class.add(Dense(256,activation=('relu')))
model_class.add(Dropout(.2))
model_class.add(Dense(197,activation=('softmax'))) #This is the classification layer

#Model summary

In [ ]:
model_class.summary()

In [ ]:
#Defining the parameters

from tensorflow.keras.optimizers import SGD

from tensorflow.keras.optimizers import RMSprop
batch_size= 100
epochs=30
learn_rate=.01

sgd=SGD(lr=learn_rate,momentum=.01,nesterov=False)


opt = RMSprop(learning_rate=1)
#Compile
model_class.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

#Training
#model_class.fit(x_train, y_train_class,validation_data =(x_test, y_test_class), epochs=100, batch_size=32)
#model_class.fit(x_test, y_test_class, epochs=100, batch_size=32)

In [ ]:
model_class.fit(x_train, y_train_class,validation_data =(x_test, y_test_class), epochs=100, batch_size=32)

In [ ]:
model_class.fit(x_train, y_train_class,validation_data =(x_test, y_test_class), epochs=200, batch_size=32)

In [ ]:
model_class.fit(x_train, y_train_class,validation_data =(x_test, y_test_class), epochs=200, batch_size=32)

In [ ]:
model_class.evaluate(x_train, y_train_class)

In [ ]:
model_class.evaluate(x_test, y_test_class)

In [ ]:
model_class.predict(x_train[10].reshape(1, 128, 128, 3))

In [ ]:
y_train_class[10]

In [ ]:
tensorflow.keras.backend.clear_session()
#del model_class
import gc
gc.collect()